<a href="https://colab.research.google.com/github/awaris123/OSNA-Project2/blob/main/GPUcolab_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install -U sentence-transformers #comment this out if you are running out of ram when using pretrained embeddings
from sentence_transformers import SentenceTransformer #comment this out if you are running out of ram when using pretrained embeddings

import numpy as np
import keras
import pandas as pd
import pdb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import importlib

using_xlnet=False
if using_xlnet:
  !pip3 install transformers
  from xlnet import XLNet

!wget "https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/data_generator.py"
!wget "https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/models/simple_classifier_model.py"
#wget xlnet if that finishes

from data_generator import DataGenerator
from simple_classifier_model import SimpleClassifier
import simple_classifier_model

from datetime import datetime
from packaging import version

import tensorflow as tf
%load_ext tensorboard

import tensorboard


Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.9)
--2020-11-29 23:23:25--  https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/data_generator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4785 (4.7K) [text/plain]
Saving to: ‘data_generator.py.6’

data_generator.py.6 100%[===================>]   4.67K  --.-KB/s    in 0s      

2020-11-29 23:23:25 (75.6 MB/s) - ‘data_generator.py.6’ saved [4785/4785]

--2020-11-29 23:23:25--  https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/models/simple_classifier_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#This is the code modified from preprocessing.py to extract embeddings from BERT or even word2vec since these vectors are wasteful to recalculate on the fly. Don't run unless you want to extract and store embeddings.




In [ ]:
mkdir ./drive/MyDrive/joint_embeddings


In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
label_encoder = LabelEncoder()
targets = label_encoder.fit_transform(df['label'].values)

X_train, X_test, y_train, y_test = train_test_split(df[['title1_en', 'title2_en']], targets, test_size=0.2)
df_train = pd.DataFrame()

params={'lr': 0.01, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 25000}

training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])
all_generator = DataGenerator(df, targets, total_samples=targets.shape[0], batch_size=params['batch_size'])
joint_embedding = np.empty((all_generator.total_samples, *all_generator.dim))
y = np.empty((all_generator.total_samples), dtype=int)

for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  e1, y1=all_generator.__getitem__(i) #

  with open(f"./joint_embedding_{i}", 'wb') as f:
    pickle.dump(e1, f)

  with open(f"./y_{i}", 'wb') as f:
    pickle.dump(y1, f)

In [ ]:
import pickle
path="./drive/MyDrive/joint_embeddings/"
batch_size=25000
X_all = np.empty((batch_size*10, 1536))
y_all = np.empty((batch_size*10))


for i in range(10):
  with open(f"{path}/joint_embedding_{i}", 'rb') as f:
    X=pickle.load(f)
    X_all[i*batch_size:(i+1)*batch_size,:]=X

  with open(f"{path}/y_{i}", 'rb') as f:
    y=pickle.load(f)
    y_all[i*batch_size:(i+1)*batch_size]=y
  print(i)



In [ ]:
with open("./joint_embedding", 'wb') as f:
  pickle.dump(X_all, f)

with open("./y", 'wb') as f:
  pickle.dump(y_all, f)

In [ ]:
!cp -r joint_embedding ./drive/MyDrive/joint_embeddings/
!cp -r y ./drive/MyDrive/joint_embeddings/

cp: cannot stat 'joint_embedding': No such file or directory
cp: cannot stat 'y': No such file or directory


In [7]:
#for xlnet

'''
training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])

if using_xlnet:
  xlnet1= XLNet()
  model = xlnet1.model
else:

model.summary()
def get_inputs(tweets, tokenizer, max_len=120):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in tweets]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

inp_tok, ids, segments = get_inputs(df[['title1_en', 'title2_en']], xlnet_tokenizer)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=params['lr'],
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=params['lr'])

model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])#, keras.metrics.Precision(), keras.metrics.Recall()])

'''
'''
model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=10,
                          workers=8,
                    use_multiprocessing=False,
                    verbose=1)

model.save("/content/drive/MyDrive/joint_embeddings/test/xlnet_ft_unbalanced")
'''

'\nmodel.fit_generator(generator=training_generator,\n                          validation_data=validation_generator,\n                          epochs=10,\n                          workers=8,\n                    use_multiprocessing=False,\n                    verbose=1)\n\nmodel.save("/content/drive/MyDrive/joint_embeddings/test/xlnet_ft_unbalanced")\n'

In [ ]:
del df
del targets

#To run simple classifier on sentenceBERT embeddings

In [1]:
#load
import pickle
path="./drive/MyDrive/joint_embeddings/"

with open(f"{path}joint_embedding", 'rb') as f:
  X_all=pickle.load(f)

with open(f"{path}y", 'rb') as f:
  y_all=pickle.load(f)

In [ ]:
#train from preprocessed data

params={'lr': 0.001, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 128}

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)
del X_all
del y_all

In [ ]:
# Define the Keras TensorBoard callback. credit to: https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/graphs.ipynb
!rm -rf ./logs/ 
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])

import importlib

importlib.reload(simple_classifier_model)

model = simple_classifier_model.SimpleClassifier(input_shape=768*2)
model.model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=params['lr'],
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=params['lr'])

model.model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])#, ])
model.model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=50,
                          workers=8,
                    use_multiprocessing=False,
                    verbose=1, callbacks=[tensorboard_callback])
model.model.save("/content/drive/MyDrive/joint_embeddings/confirm/simple")#bert_mlp_unbalanced_1500_512_bs128_leakyrelu")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 900)               1383300   
_________________________________________________________________
batch_normalization_3 (Batch (None, 900)               3600      
_________________________________________________________________
dropout_3 (Dropout)          (None, 900)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               225250    
_________________________________________________________________
batch_normalization_4 (Batch (None, 250)               1000      
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)               

#Evaluate here... Do not unless it is for Kaggle Submission. Takes a bit of time

In [ ]:
# EVALUATION
import data_generator #import DataGenerator
importlib.reload(data_generator)

#model=keras.models.load("/content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500")
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
encoder = LabelEncoder()
encoder.fit_transform(df['label'].values)

df=pd.read_csv("/content/drive/MyDrive/joint_embeddings/test.csv", sep=',')
final_df=pd.DataFrame(data=None, columns=['id', 'label'])
final_df['id'] = df['id']
targets=np.empty((final_df.shape[0]), dtype=int)
all_generator = data_generator.DataGenerator(df, targets, n_classes=3, total_samples=targets.shape[0], batch_size=128, pretrained_embeddings=False)
print(int(all_generator.total_samples/all_generator.batch_size))
for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  print(i, end = ' ')
  e1, _=all_generator.__getitem__(i) #
  targets[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=np.argmax(model.model.predict(e1), axis=1)
  print(i, end = ' ')


#label decoder
#final_df['label'] = encoder.inverse_transform(targets)

#convert to csv
#final_df.to_csv("/content/drive/MyDrive/joint_embeddings/sample_submission.csv", index=False)

500
0 0 1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8 9 9 10 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23 24 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35 36 36 37 37 38 38 39 39 40 40 41 41 42 42 43 43 44 44 45 45 46 46 47 47 48 48 49 49 50 50 51 51 52 52 53 53 54 54 55 55 56 56 57 57 58 58 59 59 60 60 61 61 62 62 63 63 64 64 65 65 66 66 67 67 68 68 69 69 70 70 71 71 72 72 73 73 74 74 75 75 76 76 77 77 78 78 79 79 80 80 81 81 82 82 83 83 84 84 85 85 86 86 87 87 88 88 89 89 90 90 91 91 92 92 93 93 94 94 95 95 96 96 97 97 98 98 99 99 100 100 101 101 102 102 103 103 104 104 105 105 106 106 107 107 108 108 109 109 110 110 111 111 112 112 113 113 114 114 115 115 116 116 117 117 118 118 119 119 120 120 121 121 122 122 123 123 124 124 125 125 126 126 127 127 128 128 129 129 130 130 131 131 132 132 133 133 134 134 135 135 136 136 137 137 138 138 139 139 140 140 141 141 142 142 143 143 144 144 145 145 146 146 147 147 148 148 149 149 150 150 151 151 

In [ ]:
len([i for i in targets if i > 2])
le_name_mapping = dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
le_name_mapping
final_df['label']=[le_name_mapping.get(i, 'unrelated') for i in targets]

#label decoder

#convert to csv
final_df.to_csv("/content/drive/MyDrive/joint_embeddings/sample_submission.csv", index=False)

# Cute Graphs and Stuff via Tensorboard

In [ ]:
%tensorboard --logdir logs